![Cartoon of telecom customers](IMG_8811.png)


The telecommunications (telecom) sector in India is rapidly changing, with more and more telecom businesses being created and many customers deciding to switch between providers. "Churn" refers to the process where customers or subscribers stop using a company's services or products. Understanding the factors that influence keeping a customer as a client in predicting churn is crucial for telecom companies to enhance their service quality and customer satisfaction. As the data scientist on this project, you aim to explore the intricate dynamics of customer behavior and demographics in the Indian telecom sector in predicting customer churn, utilizing two comprehensive datasets from four major telecom partners: Airtel, Reliance Jio, Vodafone, and BSNL:

- `telecom_demographics.csv` contains information related to Indian customer demographics:

| Variable             | Description                                      |
|----------------------|--------------------------------------------------|
| `customer_id `         | Unique identifier for each customer.             |
| `telecom_partner `     | The telecom partner associated with the customer.|
| `gender `              | The gender of the customer.                      |
| `age `                 | The age of the customer.                         |
| `state`                | The Indian state in which the customer is located.|
| `city`                 | The city in which the customer is located.       |
| `pincode`              | The pincode of the customer's location.          |
| `registration_event` | When the customer registered with the telecom partner.|
| `num_dependents`      | The number of dependents (e.g., children) the customer has.|
| `estimated_salary`     | The customer's estimated salary.                 |

- `telecom_usage` contains information about the usage patterns of Indian customers:

| Variable   | Description                                                  |
|------------|--------------------------------------------------------------|
| `customer_id` | Unique identifier for each customer.                         |
| `calls_made` | The number of calls made by the customer.                    |
| `sms_sent`   | The number of SMS messages sent by the customer.             |
| `data_used`  | The amount of data used by the customer.                     |
| `churn`    | Binary variable indicating whether the customer has churned or not (1 = churned, 0 = not churned).|


In [17]:
# Import required libraries and methods/functions
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
# OneHotEncoder is not needed if using pd.get_dummies()
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.width", 1000)

Does Logistic Regression or Random Forest produce a higher accuracy score in predicting telecom churn in India?

- Load the two CSV files into separate DataFrames. Merge them into a DataFrame named `churn_df`. Calculate the proportion of customers who have churned, and identify the categorical variables in `churn_df`.

In [11]:
telecom_demog = pd.read_csv("telecom_demographics.csv")
telecom_usage = pd.read_csv("telecom_usage.csv")

print(telecom_demog.columns)
print(telecom_usage.columns)

Index(['customer_id', 'telecom_partner', 'gender', 'age', 'state', 'city', 'pincode', 'registration_event', 'num_dependents', 'estimated_salary'], dtype='object')
Index(['customer_id', 'calls_made', 'sms_sent', 'data_used', 'churn'], dtype='object')


In [12]:
churn_df = telecom_demog.merge(telecom_usage, on='customer_id')
print(churn_df.head())

   customer_id telecom_partner gender  age             state       city  pincode registration_event  num_dependents  estimated_salary  calls_made  sms_sent  data_used  churn
0        15169          Airtel      F   26  Himachal Pradesh      Delhi   667173         2020-03-16               4             85979          75        21       4532      1
1       149207          Airtel      F   74       Uttarakhand  Hyderabad   313997         2022-01-16               0             69445          35        38        723      1
2       148119          Airtel      F   54         Jharkhand    Chennai   549925         2022-01-11               2             75949          70        47       4688      1
3       187288    Reliance Jio      M   29             Bihar  Hyderabad   230636         2022-07-26               3             34272          95        32      10241      1
4        14016        Vodafone      M   45          Nagaland  Bangalore   188036         2020-03-11               4             34

In [13]:
churn_rate = churn_df['churn'].value_counts() / len(churn_df)
print(churn_rate)

churn
0    0.799538
1    0.200462
Name: count, dtype: float64


In [14]:
print(churn_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6500 entries, 0 to 6499
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         6500 non-null   int64 
 1   telecom_partner     6500 non-null   object
 2   gender              6500 non-null   object
 3   age                 6500 non-null   int64 
 4   state               6500 non-null   object
 5   city                6500 non-null   object
 6   pincode             6500 non-null   int64 
 7   registration_event  6500 non-null   object
 8   num_dependents      6500 non-null   int64 
 9   estimated_salary    6500 non-null   int64 
 10  calls_made          6500 non-null   int64 
 11  sms_sent            6500 non-null   int64 
 12  data_used           6500 non-null   int64 
 13  churn               6500 non-null   int64 
dtypes: int64(9), object(5)
memory usage: 711.1+ KB
None


- Convert categorical features in `churn_df` into `features_scaled`. Perform feature scaling separating the appropriate features and scale them. Define your scaled features and `target` variable for the churn prediction model.

In [15]:
churn_df = pd.get_dummies(churn_df, columns=['telecom_partner', 'gender', 'state', 'city', 'registration_event'])

scaler = StandardScaler()

features = churn_df.drop(['customer_id', 'churn'], axis=1)
features_scaled = scaler.fit_transform(features)

target = churn_df['churn']

- Split the processed data into training and testing sets giving names of `X_train`, `X_test`, `y_train`, and `y_test` using an 80-20 split, setting a random state of `42` for reproducibility.

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

- Train Logistic Regression and Random Forest Classifier models, setting a random seed of `42`. Store model predictions in `logreg_pred` and `rf_pred`.

In [19]:
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)

logreg_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, logreg_pred))
print(classification_report(y_test, logreg_pred))

[[911 116]
 [243  30]]
              precision    recall  f1-score   support

           0       0.79      0.89      0.84      1027
           1       0.21      0.11      0.14       273

    accuracy                           0.72      1300
   macro avg       0.50      0.50      0.49      1300
weighted avg       0.67      0.72      0.69      1300



In [20]:
# Instantiate the Random Forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Random Forest predictions
rf_pred = rf.predict(X_test)

# Random Forest evaluation
print(confusion_matrix(y_test, rf_pred))
print(classification_report(y_test, rf_pred))

[[1026    1]
 [ 273    0]]
              precision    recall  f1-score   support

           0       0.79      1.00      0.88      1027
           1       0.00      0.00      0.00       273

    accuracy                           0.79      1300
   macro avg       0.39      0.50      0.44      1300
weighted avg       0.62      0.79      0.70      1300



- Assess the models on test data. Assign the model's name with higher accuracy (`"LogisticRegression"` or `"RandomForest"`) to `higher_accuracy`.

In [21]:
higher_accuracy = 'RandomForest'
print(higher_accuracy)

RandomForest
